In [ ]:
# 在python2將新版python3的特性import
from __future__ import division
from __future__ import absolute_import
from __future__ import print_function
from __future__ import unicode_literals

In [ ]:
# 安裝 paddle - https://www.paddlepaddle.org.cn/install/quick
# 查看版本，並到上面的網址安裝對應的版本
!python3 --version && nvcc --version
# 安裝對應版本paddle
! python3 -m pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple

Python 3.6.9
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 111.3MB 65kB/s 


In [ ]:
# 安裝ERNIE
!git clone https://github.com/PaddlePaddle/ERNIE.git
!git -C ./ERNIE branch origin/dygraph
import sys
sys.path.append('./ERNIE')
import numpy as np
from sklearn.metrics import f1_score
import paddle as P
import paddle.fluid as F
import paddle.fluid.layers as L
import paddle.fluid.dygraph as D

from ernie.tokenizing_ernie import ErnieTokenizer
from ernie.modeling_ernie import ErnieModelForSequenceClassification

Cloning into 'ERNIE'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 1880 (delta 1), reused 0 (delta 0), pack-reused 1873
Receiving objects: 100% (1880/1880), 88.77 MiB | 36.63 MiB/s, done.
Resolving deltas: 100% (1023/1023), done.


In [ ]:
# 下載資料
! wget https://ernie-github.cdn.bcebos.com/data-chnsenticorp.tar.gz
! tar -xvzf data-chnsenticorp.tar.gz
! rm data-chnsenticorp.tar.gz

--2020-10-25 09:27:57--  https://ernie-github.cdn.bcebos.com/data-chnsenticorp.tar.gz
Resolving ernie-github.cdn.bcebos.com (ernie-github.cdn.bcebos.com)... 125.77.154.35
Connecting to ernie-github.cdn.bcebos.com (ernie-github.cdn.bcebos.com)|125.77.154.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1753724 (1.7M) [application/x-gzip]
Saving to: ‘data-chnsenticorp.tar.gz’

data-chnsenticorp.t 100%[===================>]   1.67M  1.36MB/s    in 1.2s    

2020-10-25 09:28:00 (1.36 MB/s) - ‘data-chnsenticorp.tar.gz’ saved [1753724/1753724]

chnsenticorp/
chnsenticorp/dev/
chnsenticorp/dev/part.0
chnsenticorp/test/
chnsenticorp/test/part.0
chnsenticorp/train/
chnsenticorp/train/part.0


In [ ]:
# 查看訓練文件
! head -n 5 chnsenticorp/train/part.0

选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般	1
15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错	1
房间太小。其他的都一般。。。。。。。。。	0
1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.	0
今天才知道这书还有第6卷,真有点郁闷:为什么同一套书有两种版本呢?当当网是不是该跟出版社商量商量,单独出个第6卷,让我们的孩子不会有所遗憾。	1


In [ ]:
# parameter setting
BATCH=32
MAX_SEQLEN=300
LR=5e-5
EPOCH=3

In [ ]:
# 建立tokenizer，並下載pretrain好的參數
tokenizer = ErnieTokenizer.from_pretrained('ernie-1.0')
def make_data(path):
    data = []
    for i, l in enumerate(open(path)):
        if i == 0:
            continue
        l = l.strip().split('\t')
        # 修改一下parser
        text, label = l[0], int(l[-1])
        text_id, _ = tokenizer.encode(text) # 給每個word一個獨立的ID，包含一些特殊的tag SEP,CLS
        text_id = text_id[:MAX_SEQLEN] # 句子長超過MAX_SEQLEN截斷
        text_id = np.pad(text_id, [0, MAX_SEQLEN-len(text_id)], mode='constant') # 不足300補到300
        label_id = np.array(label+1) # 1=>neg 2=>pos
        data.append((text_id, label_id))
    return data

downloading https://ernie-github.cdn.bcebos.com/model-ernie1.0.1.tar.gz: 788478KB [00:56, 13919.41KB/s]                            


In [ ]:
# 讀訓練資料
train_data = make_data('chnsenticorp/train/part.0')
# 讀測試資料
test_data = make_data('chnsenticorp/test/part.0')

In [ ]:
# 讓Paddle進入動態圖，這好像是Paddle提供的功能，不過沒怎麼用過
# 查了一下像tensorflow這種需要先建立完模型架構，之後行程式才可以拿到結果的稱為靜態圖
# 動態圖則可以在程式中很方便的使用python直接拿到數值，在寫程式上方便很多
# https://blog.csdn.net/PaddlePaddle/article/details/100059492
D.guard().__enter__() 

ernie = ErnieModelForSequenceClassification.from_pretrained('ernie-1.0', num_labels=3)
optimizer = F.optimizer.Adam(LR, parameter_list=ernie.parameters())

def get_batch_data(data, i):
    d = data[i*BATCH: (i + 1) * BATCH]
    feature, label = zip(*d)
    feature = np.stack(feature)  # 将BATCH行样本整合在一个numpy.array中
    label = np.stack(list(label))
    print(label)
    print(feature)
    feature = D.to_variable(feature) # 使用to_variable将numpy.array转换为paddle tensor
    label = D.to_variable(label)
    return feature, label

for i in range(EPOCH):
    np.random.shuffle(train_data) # 每个epoch都shuffle数据以获得最佳训练效果；
    #train
    for j in range(len(train_data) // BATCH):
        feature, label = get_batch_data(train_data, j)
        loss, _ = ernie(feature, labels=label) # 回傳(loss, logits)； loss為目前模型的loss, logits為最後一層的output
        loss.backward()
        optimizer.minimize(loss)
        ernie.clear_gradients()
        # 每十個batch查看loss
        if j % 10 == 0:
            print('train on epoch %d batch %d: loss %.5f' % (i, j, loss.numpy()))
    # evaluate
    all_pred, all_label = [], []
    with D.base._switch_tracer_mode_guard_(is_train=False): # 在这个with域内ernie不会进行梯度计算；
        ernie.eval() # 進入 eval 關閉 drop out
        for j in range(len(test_data) // BATCH):
            feature, label = get_batch_data(test_data, j)
            # logits => 最後一層的output
            loss, logits = ernie(feature, labels=label) 
            # L.argmax(logits, -1) => 拿機率最大的index
            all_pred.extend(L.argmax(logits, -1).numpy())
            all_label.extend(label.numpy())
        ernie.train() # 進入 train 模式
    f1 = f1_score(all_label, all_pred, average='macro')
    print('f1 %.5f' % f1)

param:classifier.weight not set in pretrained model, skip
param:classifier.bias not set in pretrained model, skip
/usr/local/lib/python3.6/dist-packages/paddle/fluid/dygraph/layers.py:718: UserWarning: Varibale [ mlm_bias mlm.weight mlm.bias mlm_ln.weight mlm_ln.bias ] are not used, because not included in layers state_dict
  format(" ".join(unused_para_list)))


## test

In [ ]:
def tokenize_eval_data(text):
    text_id, _ = tokenizer.encode(text) # 給每個word一個獨立的ID，包含一些特殊的tag SEP,CLS
    text_id = text_id[:MAX_SEQLEN] # 句子長超過MAX_SEQLEN截斷
    text_id = np.pad(text_id, [0, MAX_SEQLEN-len(text_id)], mode='constant') # 不足300補到300
    return text_id

def get_eval_data(text, label):
    feature = np.stack([text,text])
    print(feature)
    feature = D.to_variable(feature) # 使用to_variabler將numpy.array轉換為paddle tensor
    print(feature)
    label = np.stack([label])
    print(label)
    label = D.to_variable(label)
    return feature, label

In [ ]:
# test data
test1 = "拜登在搖擺州賓州的造勢活動上批評川普說，「他在中國貿易問題上一直軟弱又混亂，所做一切只是為大型銀行打開在中國賺錢的大門，並幫忙造成美國農場破產數量的激增」，並稱自己將做到川普無法做到的事情。"
label1 = 1
# preprosee
test1 = tokenize_eval_data(test1)
test1,labe1 = get_eval_data(test1, label1)
# ernie model predict
ernie.eval()
loss, logits = ernie([np.array(tokenizer.encode(test1))], labels=labe1)

NameError: ignored

In [ ]:
ernie.eval()
# test
test1 = "我喜歡"
tokenizer.encode(test1)
loss, logits = ernie([np.array(tokenizer.encode(test1))], labels=2)
# print(logits)